In [34]:
from datasets import load_from_disk

In [ ]:
# run this cell two times: one with kyc and one with crm dataset
dataset=load_from_disk('dataset/crm-huggingface-dataset')
dataset

In [ ]:
dataset = dataset.remove_columns(["locale", "client_id"])

print(dataset)

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium",cache_dir='v3')

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="Persian", task="transcribe",cache_dir='v3')

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="Persian", task="transcribe",cache_dir='v3')

In [29]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [30]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [31]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=10)

Map (num_proc=10):  28%|██▊       | 215852/780809 [3:02:46<8:28:11, 18.53 examples/s] 

In [37]:
dataset.save_to_disk('dataset/whisper_processed_data/crm-processed.hf')

Saving the dataset (167/167 shards): 100%|██████████| 86757/86757 [04:28<00:00, 323.05 examples/s]


In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 780809
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 86757
    })
})

In [38]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 780809
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 86757
    })
})

In [45]:
dataset_filtered=dataset['test'].select(range(5000,7500)).filter(lambda x: len(x['labels'])==0)
dataset_filtered

Filter: 100%|██████████| 2500/2500 [04:16<00:00,  9.76 examples/s]


Dataset({
    features: ['input_features', 'labels'],
    num_rows: 0
})

In [46]:
dataset_filtered

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 0
})